In [ ]:
from nbdev import *

In [ ]:
#default_exp sample

# Sample

> Create samples from Newspaper navigator dataset

\ # TODO Intro to module 

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#export
from nnanno.core import *

In [ ]:
#export
# TODO tidy imports

# sys 
import io
import shutil
import pkg_resources
from pathlib import Path
from datetime import datetime

# other
from tqdm.auto import trange, tqdm
import requests
import ijson
import functools
import math
from cytoolz import dicttoolz, itertoolz
import random
import json
from PIL import Image
import PIL
import concurrent.futures
import numpy as np
import itertools
from pandas import json_normalize
import pandas as pd
from functools import partial
import numpy as np

# typing
from typing import (
    Any,
    Optional,
    Union,
    Dict,
    List,
    Tuple,
    Set,
    Iterable,
)

In [ ]:
#export
def get_json_url(year: Union[str,int], kind:str='photos') -> str:
    '''Returns url for the json data from news-navigator for given `year` and `kind`'''
    return f'https://news-navigator.labs.loc.gov/prepackaged/{year}_{kind}.json'

In [ ]:
assert get_json_url(1860) == 'https://news-navigator.labs.loc.gov/prepackaged/1860_photos.json' 
assert get_json_url(1950) == 'https://news-navigator.labs.loc.gov/prepackaged/1950_photos.json' 
assert get_json_url(1950,'ads') == 'https://news-navigator.labs.loc.gov/prepackaged/1950_ads.json' 

In [ ]:
#export
def load_json(url) -> Dict[str, Any]:
    """Returns loaded json from url

    Parameters:
    url (str): URL for news-navigator json file

    Returns:
    Dict: dictionary with data from input json url
    """
    with requests.get(url, timeout=2) as r:
        r.raise_for_status()
        return json.loads(r.content)

In [ ]:
test_json = load_json('https://news-navigator.labs.loc.gov/prepackaged/1950_photos.json')
assert type(test_json[0]) == dict
assert type(test_json) == list

This works well for a smallish file but if we try this with the [1905_ads.json](https://news-navigator.labs.loc.gov/prepackaged/1910_ads.json) file which is ~3.3GB we will likely run out of memory. For example running 

```python
with requests.get('https://news-navigator.labs.loc.gov/prepackaged/1910_ads.json') as r:
    data = json.loads(r.content)
len(data)
```

on a Google Colab instance with 25GB of RAM causes a crash. 

In [ ]:
#export
@functools.lru_cache(256)
def count_json_iter(url: str, session=None) -> int:
    """
    Returns count of objects in url json file using an iterator to avoid loading json          into memory

    Parameters:
    url (str): URL for news-navigator json file

    Returns:
    int: count of json objects in url
    """
    if not session:
        session = create_cached_session()
    with session.get(url, timeout=60) as r:
        r.raise_for_status()
        if r:
            objects = ijson.items(r.content, "item")
            count = itertoolz.count(iter(objects))
        else:
            count = np.nan
    return count

In [ ]:
count_json_iter('https://news-navigator.labs.loc.gov/prepackaged/1850_photos.json')

22

In [ ]:
url = 'https://news-navigator.labs.loc.gov/prepackaged/1850_photos.json'
assert type(count_json_iter(url)) == int
assert len(json.loads(requests.get(url).content)) == count_json_iter(url)

In [ ]:
#export
@functools.lru_cache(256)
def get_year_size(year,kind):
    session = None
    dset_size = {}       
    url = get_json_url(year,kind)
    if kind == ('ads' or 'headlines') and int(year) >=1870:
        session = create_session()
    dset_size[str(year)] = count_json_iter(url, session)
    return dset_size

In [ ]:
get_year_size(1850, 'photos')

{'1850': 22}

In [ ]:
#export
@functools.lru_cache(512)
def get_year_sizes(kind,start=1850, end=1950, step=5):
    """
    Returns the sizes for json data files for `kind` between year `start` and `end`    
    with step size 'step'

    Parameters:
    kind (str): kind of image from news-navigator options:
    photos, illustrations, maps, comics, cartoons, headlines, ads

    Returns:
    Pandas.DataFrame: with data from input json url
    """
  #  dset_size = {}
    futures = []
    years = range(start,end+1,step)
    max_workers = get_max_workers(years)
    with tqdm(total=len(years)) as progress:
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            for year in years:
                future = executor.submit(get_year_size, year, kind)
                future.add_done_callback(lambda p: progress.update())
                futures.append(future)
        results = [future.result() for future in futures]
        dset_size = {k: v for d in results for k, v in d.items()}
    year_df = pd.DataFrame.from_dict(dset_size,orient='index',columns=[f'{kind}_count'])
    return year_df


Returns the year sizes for a given kind taking a step size `step`. For example to get the number of photos in the news-navigator dataset between 1850 and 1860 for every year:

In [ ]:
%%time
get_year_sizes('photos',1850, 1860, step=1)

100%|██████████| 11/11 [00:00<00:00, 134.02it/s]CPU times: user 52.4 ms, sys: 53.6 ms, total: 106 ms
Wall time: 121 ms



,photos_count
1850,22
1851,20
1852,22
1853,45
1854,221
1855,17
1856,49
1857,74
1858,39
1859,28


In [ ]:
assert len(get_year_sizes('photos',1850, 1860, step=1)) == 11
assert len(get_year_sizes('photos',1850,1860, step=2)) == 6

100%|██████████| 6/6 [00:00<00:00, 5343.06it/s]


In [ ]:
#export
def get_all_year_sizes(start=1850, end=1950,step=1, save:bool=True):
    """
    Returns a dataframe with number of counts from year `start` to `end`
    """
    kinds = ['ads', 'photos', 'maps', 'illustrations', 'comics', 'cartoons','headlines']
    dfs = []
    for kind in tqdm(kinds):
        df = get_year_sizes(kind, start=start, end=end,step=step)
        dfs.append(df)
    df = pd.concat(dfs, axis=1)
    df['total'] = df.sum(axis=1)
    if save:
        df.to_csv('all_year_sizes.csv')
    return df

## Sampling 

### Streaming sampling

Since we want a subset of the Newspaper Navigator datasets which we can either work with for [annotation](!TODO add link) or for inference we want to create samples. Sampling in python can be complicated depending on the type of population you are working with and the properties your sample needs to have but usually we can do something fairly simple like. For example, if we want to sample from a selection of books we could do:

In [ ]:
import random
books = ['War and Peace', 'Frankenstein', 'If They Come in the Morning']
random.sample(books, 1)

['Frankenstein']

However, we run into a same problem as when trying to get the length of a json dataset which wouldn't fit into memory above. For example if we want to sample $k$ examples from one of our json files which we can't load into memory. To get around this we can use [Reservoir_sampling](https://en.wikipedia.org/wiki/Reservoir_sampling):

> Reservoir sampling is a family of randomized algorithms for choosing a simple random sample without replacement of k items from a population of unknown size n in a single pass over the items. The size of the population n is not known to the algorithm and is typically too large to fit all n items into main memory. The population is revealed to the algorithm over time, and the algorithm cannot look back at previous items. 



In [ ]:
#export
def sample_stream(stream, k:int):
    """
    Return a random sample of k elements drawn without replacement from stream.
    Designed to be used when the elements of stream cannot easily fit into memory.
    """
    r = np.array(list(itertools.islice(stream, k)))
    for t, x in enumerate(stream, k + 1):
        i = np.random.randint(1, t + 1)

        if i <= k:
            r[i - 1] = x
    return r

Now we sample whilst only loading a small numer of items into memory at one time. This does come at some cost, mainly speed. There are faster ways of sampling from a stream but this isn't the main bottle neck for sampling in this case. 

In [ ]:
sample_stream(range(1,100000), 5)

array([42723,  1354, 86457, 75827, 75402])

In [ ]:
names = ['Karl Marx', 'Rosa Luxenburg', 'Raya Dunayevskaya', 'CLR James']
sample_stream(iter(names), 2)

array(['Raya Dunayevsk', 'Rosa Luxenburg'], dtype='<U14')

In [ ]:
#hide
assert len(sample_stream(range(1,100),5)) == 5

In [ ]:
#export
@functools.lru_cache(128)
def calc_frac_size(url,frac):
    "returns fraction size from a json stream"
    return round(count_json_iter(url)*frac)

In [ ]:
#hide
url = get_json_url(1850)
assert calc_frac_size(url, 0.5)== 11 #22*0.5

In [ ]:
#export
def calc_year_from_total(total,start,end,step):
    "Calculate size of a year sample based on a total sample size"
    return max(1,round(total/(((end-start)+1)/step)))

In [ ]:
calc_year_from_total(10,1850, 18950,1)

1

In [ ]:
#hide 
assert calc_year_from_total(10, 1850,1950,1) >=1 # test that a value is always returned 

In [ ]:
#export
def reduce_df_memory(df):
    return df.astype(
            {"score": "float64",
                "page_seq_num": "int32",
                "batch": "category",
                "box":"object",
                "lccn": "category",
                "page_url": "category",
                "name": "category",
                "publisher": "category",
                "place_of_publication": "category",
                "edition_seq_num": "category"}
        )

In [ ]:
# export
def sample_year(kind:str,sample_size:Union[int,float], year:int) ->np.array:
    url = get_json_url(year, kind)
    if type(sample_size) is float:
        sample_size = calc_frac_size(url, sample_size)
    if kind == ('ads' or 'headlines') and int(year) >=1870:
        session = create_session()
    else:
        session = create_cached_session()
    with session.get(get_json_url(year, kind)) as r:
        if r:
            try:
                data = ijson.items(r.content, "item")
                sample_data = sample_stream(iter(data), sample_size)
            except requests.exceptions.RequestException as e:
                sample_data = np.nan
    return sample_data

In [ ]:
sample_year('photos', 1, 1850)

array([{'filepath': 'ohi_ingstad_ver01/data/sn85026051/00296027029/1850121401/0120/000_0_93.jpg', 'pub_date': '1850-12-14', 'page_seq_num': 120, 'edition_seq_num': 1, 'batch': 'ohi_ingstad_ver01', 'lccn': 'sn85026051', 'box': [Decimal('0.3187102596316717'), Decimal('0.6281719624215338'), Decimal('0.4516467879303789'), Decimal('0.7379311907239355')], 'score': Decimal('0.9390530586242676'), 'ocr': ['COME', 'IN,', 'WE', 'CALL', 'YOU!'], 'place_of_publication': 'Fremont, Sandusky County, Ohio', 'geographic_coverage': ['Ohio--Sandusky--Fremont'], 'name': 'Fremont weekly freeman. [volume]', 'publisher': 'J.S. Fouke', 'url': 'https://news-navigator.labs.loc.gov/data/ohi_ingstad_ver01/data/sn85026051/00296027029/1850121401/0120/000_0_93.jpg', 'page_url': 'https://chroniclingamerica.loc.gov/data/batches/ohi_ingstad_ver01/data/sn85026051/00296027029/1850121401/0120.jp2'}],
      dtype=object)

In [ ]:
#export
class nnSampler:
    def __init__(self):
        self.population = pd.read_csv(pkg_resources.resource_stream('nnanno', 'data/all_year_counts.csv'), 
                                      index_col=0)
    
    def __repr__(self):
        return (f'{self.__class__.__name__}')
        

    def create_sample(
        self,
        sample_size: Union[int, float],
        kind: str = "photos",
        start_year: int = 1850,
        end_year: int = 1950,
        step: int = 5,
        year_sample=True,
        save: bool = False,
        reduce_memory=True,
    ):
        if not year_sample:
            if not type(sample_size) == int:
                raise ValueError(
                    f"type{sample_size} is not an int. Fractions are only supported for sampling by year"
                )
            sample_size = calc_year_from_total(sample_size, start_year, end_year, step)
        futures = []
        years = range(start_year, end_year + 1, step)
        _year_sample = partial(sample_year, kind, sample_size)
        with tqdm(total=len(years)) as progress:
            with concurrent.futures.ThreadPoolExecutor(2) as executor:
                for year in years:
                    future = executor.submit(_year_sample, year)
                    future.add_done_callback(lambda p: progress.update())
                    futures.append(future)
        results = [future.result() for future in futures]
        df = pd.DataFrame.from_dict(list(itertoolz.concat(results)))

        if reduce_memory:
            df = reduce_df_memory(df)
        if save:
            df.to_json(f"{kind}_{start_year}_{end_year}_sample.json")
        self.sample = df
        return df

    def download_sample(
        self,
        out_dir,
        csv_name=None,
        df=None,
        original: bool = True,
        pct: int = None,
        size: tuple = None,
        preserve_asp_ratio: bool = True,
    ):
        if df is not None:
            self.download_df = df.copy(deep=True)
        else:
            try:
                self.download_df = self.sample.copy(deep=True)
            except AttributeError as E:
                print(
                    "You need to create a sample before downloading, or pass in a previously created "
                )
        self.download_df["iiif_url"] = self.download_df.apply(
            lambda x: iif_df_apply(
                x,
                original=original,
                pct=pct,
                size=size,
                preserve_asp_ratio=preserve_asp_ratio,
            ),
            axis=1,
        )
        self.download_df["download_image_path"] = self.download_df['filepath'].str.replace('/','_')
        
        if not Path(out_dir).exists():
            Path(out_dir).mkdir(parents=True)
        _download_image = lambda x: download_image(
            x.iiif_url, x.download_image_path, out_dir)
        with tqdm(total=len(self.download_df)) as progress:
            with concurrent.futures.ThreadPoolExecutor() as executor:
                futures = []
                for tuple_row in self.download_df.itertuples():
                    future = executor.submit(_download_image, tuple_row)
                    future.add_done_callback(lambda p: progress.update())
                    futures.append(future)
                del futures
        if csv_name is None:
            today = datetime.today()
            time_stamp = today.strftime("%Y_%d_%m_%H_%M")
            csv_name = f"{time_stamp}_{len(self.download_df)}_sample"
        #self.download_df.to_csv(f'{out_dir}/{csv_name}.csv')
        self.download_df.to_json(f'{out_dir}/{csv_name}.json') # TODO make sure to use json for saving outputs of samples

In [ ]:
sampler = nnSampler()

In [ ]:
sampler

nnSampler

In [ ]:
sampler.population.sum()

ads_count               63338192
photos_count             1543525
maps_count                200188
illustrations_count       798475
comics_count              526319
cartoons_count            206054
headlines_count         34155014
total                  100767767
dtype: int64

In [ ]:
df = sampler.create_sample(sample_size=10, kind='ads', start_year=1850,end_year=1860,reduce_memory=True)
df.head(5)

100%|██████████| 3/3 [00:02<00:00,  1.05it/s]


,filepath,pub_date,page_seq_num,edition_seq_num,batch,lccn,box,score,ocr,place_of_publication,geographic_coverage,name,publisher,url,page_url
0,pst_intramural_ver01/data/sn83032276/002960285...,1850-04-26,215,1,pst_intramural_ver01,sn83032276,"[0.7893347074607134, 0.5260661138779277, 0.944...",0.923698,"[j., w,, PISIITeKJ, Attorney, at, L, n, w,, LE...","Lewistown, Pa.",[Pennsylvania--Mifflin--Lewistown],Lewistown gazette. [volume],William Ross,https://news-navigator.labs.loc.gov/data/pst_i...,https://chroniclingamerica.loc.gov/data/batche...
1,msar_ikat_ver01/data/sn87090283/00199917365/18...,1850-01-18,194,1,msar_ikat_ver01,sn87090283,"[0.7196589680100182, 0.9352155295751428, 0.851...",0.943004,"[Salt., mr, SACKS, SALT—Just, received, per, f...","Port Gibson, Claiborne Co., Miss.",[Mississippi--Claiborne--Port Gibson],"The Port Gibson herald, and correspondent.",Wm. F. Eisely,https://news-navigator.labs.loc.gov/data/msar_...,https://chroniclingamerica.loc.gov/data/batche...
2,dlc_dragon_ver01/data/sn82014434/00415661381/1...,1850-09-24,315,1,dlc_dragon_ver01,sn82014434,"[0.055765863577795595, 0.9405716126223644, 0.1...",0.901824,"[OK, HURCBVI, WRITIBOB., *'/\PITM, EATER"", and...",Washington [D.C.],[District of Columbia--Washington],The republic. [volume],Gideon & Co.,https://news-navigator.labs.loc.gov/data/dlc_d...,https://chroniclingamerica.loc.gov/data/batche...
3,ohi_carrie_ver01/data/sn88078466/00279556161/1...,1850-07-10,621,1,ohi_carrie_ver01,sn88078466,"[0.6551259316584248, 0.8280968066708592, 0.799...",0.907233,"[r5, i, s, e, n, S, o, von, joiix, TEai.\'is,,...","Canton, Stark County, Ohio",[Ohio--Stark--Canton],Ohio Staats-bote.,D.F. Rothnagel,https://news-navigator.labs.loc.gov/data/ohi_c...,https://chroniclingamerica.loc.gov/data/batche...
4,ncu_jefferson_ver02/data/sn84020750/0041566727...,1850-05-11,507,1,ncu_jefferson_ver02,sn84020750,"[0.6733089634841893, 0.2027780181986831, 0.826...",0.937159,"[TIN, WARE,, At, Wholesale, and, Retail., F., ...",Fayetteville [N.C.],[North Carolina--Cumberland--Fayetteville],The North-Carolinian. [volume],H.L. Holmes,https://news-navigator.labs.loc.gov/data/ncu_j...,https://chroniclingamerica.loc.gov/data/batche...


In [ ]:
df.to_json('data.json')

### Downloading a sample 

In [ ]:
df.to_json('test.json')

In [ ]:
sampler.create_sample(sample_size=10, kind='ads', start_year=1900,end_year=1920,reduce_memory=False)
sampler.download_sample('test_iif',pct=50)

  0%|          | 0/5 [00:07<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
#hide
sampler = nnSampler()
sampler.create_sample(5, step=50)
sampler.download_sample('test_iif',pct=5)
files = [f for f in Path('test_iif').iterdir()]
assert type(Image.open(files[0])) == PIL.JpegImagePlugin.JpegImageFile
shutil.rmtree('test_iif')

100%|██████████| 15/15 [00:00<00:00, 15.86it/s]


In [ ]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_sample.ipynb.
Converted 02_annotate.ipynb.
Converted index.ipynb.
